In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import wandb
from catboost import CatBoostClassifier, Pool

from tqdm.auto import tqdm


In [ ]:
# set globally here
exchange = "ise"  # "cboe"
strategy = "supervised"  # "transfer"
subset = "test"  # "test"

retrain = True

# ise, supervised
models = [
    ("classical", "17malsep_CatBoostClassifier_default.cbm:v7"),
    ("classical-size", "3laathab_CatBoostClassifier_default.cbm:v7"),
    ("ml", "2a9iqsn0_CatBoostClassifier_default.cbm:v4"),
    ("semi-classical", "3dtuffae_CatBoostClassifier_default.cbm:v7"),
    ("semi-classical-size", "2vnqg1sj_CatBoostClassifier_default.cbm:v8"),
]

# cboe, supervised, semi-sueprvised
# models = [
#     ("classical", "30sl6vqf_CatBoostClassifier_default.cbm:v5"),
#     ("classical-size", "2w28suql_CatBoostClassifier_default.cbm:v7"),
#     ("ml", "2zglfgfb_CatBoostClassifier_default.cbm:v4"),
# ]


In [ ]:
# key used for files and artefacts
key = f"{exchange}_gbm_{strategy}_{subset}"
dataset = f"fbv/thesis/{exchange}_{strategy}_log_standardized:latest"


In [ ]:
features_option = [
    "STRK_PRC",
    "ttm",
    "option_type",
    "issue_type",
    "root",
    "myn",
    "day_vol",
]

# https://github.com/KarelZe/thesis/blob/main/notebooks/
# 3.0a-mb-explanatory_data_analysis.ipynb
features_categorical = [
    ("root", 8667),
    ("option_type", 2),
    ("issue_type", 6),
]

features_classical = [
    "TRADE_PRICE",
    "bid_ex",
    "ask_ex",
    "BEST_ASK",
    "BEST_BID",
    "price_ex_lag",
    "price_ex_lead",
    "price_all_lag",
    "price_all_lead",
    "chg_ex_lead",
    "chg_ex_lag",
    "chg_all_lead",
    "chg_all_lag",
    "prox_ex",
    "prox_best",
]

features_size = [
    "bid_ask_size_ratio_ex",
    "rel_bid_size_ex",
    "rel_ask_size_ex",
    "TRADE_SIZE",
    "bid_size_ex",
    "ask_size_ex",
    "depth_ex",
]

features_classical_size = [
    *features_classical,
    *features_size,
]

features_ml = [*features_classical_size, *features_option]


In [ ]:
os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"


In [ ]:
# see https://wandb.ai/fbv/thesis/runs/kwlaw02g/overview?workspace=user-karelze
run = wandb.init(project="thesis", entity="fbv")

artifact = run.use_artifact(dataset)
data_dir = artifact.download()


In [ ]:
if subset == "all":
    train = pd.read_parquet(Path(data_dir, "train_set.parquet"), engine="fastparquet")
    val = pd.read_parquet(Path(data_dir, "val_set.parquet"), engine="fastparquet")
    test = pd.read_parquet(Path(data_dir, "test_set.parquet"), engine="fastparquet")
    data = pd.concat([train, val, test])

elif subset == "test":
    data = pd.read_parquet(Path(data_dir, "test_set.parquet"), engine="fastparquet")

y_test = data["buy_sell"]
X_test = data.drop(columns="buy_sell")
timestamp_test = np.linspace(0, 1, len(y_test))


if retrain:
    train = pd.read_parquet(Path(data_dir, "train_set.parquet"), engine="fastparquet")
    val = pd.read_parquet(Path(data_dir, "val_set.parquet"), engine="fastparquet")
    retrain_data = pd.concat([train, val])
    y_retrain = retrain_data["buy_sell"]
    X_retrain = retrain_data.drop(columns="buy_sell")

    weight_retrain = np.geomspace(0.001, 1, num=len(y_retrain))
    timestamp_retrain = np.linspace(0, 1, len(y_retrain))


## CatBoost Baseline 🐈‍⬛

In [ ]:
results = []

feature_map = {
    "classical": features_classical,
    "classical-size": features_classical_size,
    "ml": features_ml,
    "semi-classical": features_classical,
    "semi-classical-size": features_classical_size,
    "semi-ml": features_ml,
}

for feature_str, model in tqdm(models):

    model_name = model.split("/")[-1].split(":")[0]

    artifact = run.use_artifact(model)
    model_dir = artifact.download()

    model = CatBoostClassifier()
    model.load_model(fname=Path(model_dir, model_name))

    fs = feature_map.get(feature_str)
    # filter categorical features that are in subset and get cardinality
    cat_features_sub = [tup[0] for tup in features_categorical if tup[0] in fs]

    test_pool = Pool(
        data=X_test.loc[:, fs],
        label=y_test,
        cat_features=cat_features_sub,
        timestamp=timestamp_test,
    )
    result = pd.Series(
        data=model.predict(test_pool),
        index=X_test.index,
        name=f"gbm({feature_str})",
    )
    results.append(result)

    # retrain on training and validation set
    if retrain and not feature_str.startswith("semi")::
        retrain_pool = Pool(
            data=X_retrain.loc[:, fs],
            label=y_retrain,
            cat_features=cat_features_sub,
            weight=weight_retrain,
            timestamp=timestamp_retrain,
        )

        model.fit(retrain_pool, verbose=False)
        result = pd.Series(
            data=model.predict(test_pool),
            index=X_test.index,
            name=f"gbm({feature_str}-retraining)",
        )
        results.append(result)


In [ ]:
results = pd.concat(results, axis=1)
output_path = (
    f"gs://thesis-bucket-option-trade-classification/data/results/{key}.parquet"
)
results.to_parquet(output_path)

# Log the artifact to save it as an output of this run
result_set = wandb.Artifact(name=key, type="results")
result_set.add_reference(output_path, name="results")
run.log_artifact(result_set)

wandb.finish()


In [ ]:
results.columns
